# Calchylus - Lambda calculus in Hy

In [1]:
(eval-and-compile
 
  (setv comma '·)
 
  (defn drop-until [x l] 
    (do
      (try 
        (setv y (.index l x))
        (except [ValueError] (setv y -1)))
      (cut l (inc y))))

  (defn take-until [x l] 
    (do
      (try 
        (setv y (.index l x))
        (except [ValueError] (setv y (len l))))
      (cut l 0 y)))

  ;(defn hcdot [expr] 
  ;  (list 
  ;    (map (fn [x] (if (= x hy.HyCons) comma x)) expr)))
 
  (defn args [expr]
    (if (= (first expr) comma) [] (take-until comma expr))))

(defmacro 𝜆 [&rest expr]
  `(fn [~@(args expr)] ~(first (drop-until comma expr))))

;(defsharp L [expr]
;  (do
;    (print (first expr))
;    (setv expr (hcdot expr))
;    (print
;      expr
;      (drop-until comma `~expr) 
;      (take-until comma `~expr))
;    `(fn ~(first expr) ~(second expr))))

; Ÿ Application
;(defmacro Ÿ [func &rest args] (print `(apply ~func ~args)))
; 𝜆 lambda function / expression / abstraction
;(defmacro λ [&rest expr] (print expr))

<function <lambda> at 0x00000263475349D8>

#### Terms

In [2]:
; Terms: [𝜆 x .]
; Abstraction: (𝜆 x . x) -> creates a definition of (fn [x] x) which is the identity function that is supposed to return the same value as passed
; Application: ((𝜆 x . x) 1) -> calls for identity function with a value 1, so the output / value of this application is 1

#### Lambda abstraction
$(𝜆 \space x · x)$

In [3]:
(macroexpand '(𝜆 x · x))

('fn' ['x'] 'x')

#### Lambda application

$((𝜆 \space x · x)\space y)$

#### Identity function

Function returns same value that is passed as an argument:

In [4]:
((𝜆 x · x) 1)

1

#### Function as an argument

In [5]:
(((𝜆 x · x) (𝜆 x · x)) 1)

1

#### Self application


In [6]:
(((𝜆 x · (x x)) (𝜆 x · x)) 1)

1

#### Apply function

Further abstracting functional properties of the lambda expressions, we can imitate implement (apply func args) with the next statement:

In [7]:
(((𝜆 func · (𝜆 arg · (func arg))) (𝜆 x · x)) 2)

2

In [8]:
;λfunc.λarg.(func arg)
;((𝜆 x · (* x ((𝜆 x · x) 2))) 2)

In [9]:
;((λfirst.λsecond.first x) y)
(((𝜆 x · (𝜆 y · (* y x))) 2) 2)

4

#### Constants

Whatever is the argument, return 1:

In [10]:
((𝜆 x · 2) 1)

2

In [11]:
(, ((𝜆 x · x) 1) ((𝜆 x · x) 2) ((𝜆 x · x) 3))

(1, 2, 3)

#### Infinite recursion

Produce infinite loop:

In [12]:
(try
 ; we will catch recursion error from this line, otherwise output would be messier
 ((𝜆 x · (x x)) (𝜆 x · (x x)))
 (except (e RecursionError)
   (print "Eternal loop achieved!")))

Eternal loop achieved!


#### No arguments?!

In original Lambda Calculus functions must have one and only one argument. Lambda macro in Calchylus is not that restricted. We can omit argument for simplicity:

In [13]:
((𝜆 · 2))

2

#### Many arguments

Also multiple arguments can be passed to the lambda function althought the original specification allows one and only one argument to be passed to the Lambda function.

Support for both zero and multiple arguments simpifies the Lambda expression notation.

In [14]:
((𝜆 x y · (+ x y)) 1 2)

3